In [9]:
import pandas as pd
import csv
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [8]:
def csv_to_json(csv_file_path, json_file_path):
    data = []

    # Lire le fichier CSV
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        
        # Convertir chaque ligne en un dictionnaire et l'ajouter à la liste des données
        for row in csv_reader:
            data.append(row)
    
    # Écrire les données dans un fichier JSON
    with open(json_file_path, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

# Chemins des fichiers CSV et JSON
ability_csv = 'Abilities Table.csv'
ability_mechanics_csv = 'Ability mechanics.csv'
item_tables_consumables_noncombat_items_csv = 'Item Tables - Consumables and noncombat items.csv'
item_tables_skyship_upgrades_csv = 'Item Tables - Skyship Upgrades.csv'
Item_Tables_WEAPONS = 'Item Tables - WEAPONS.csv'

ability_json = 'json_files/Abilities Table.json'
ability_mechanics_json = 'json_files/Ability mechanics.json'
item_tables_consumables_noncombat_items_json = 'json_files/item_tables_consumables_noncombat_items.json'
item_tables_skyship_upgrades_json = 'json_files/item_tables_skyship_upgrades.json'
Item_Tables_WEAPONS_json = 'json_files/Item Tables - WEAPONS.json'


# Conversion des fichiers CSV en JSON
csv_to_json(ability_csv, ability_json)
csv_to_json(ability_mechanics_csv, ability_mechanics_json)
csv_to_json(item_tables_consumables_noncombat_items_csv, item_tables_consumables_noncombat_items_json)
csv_to_json(item_tables_skyship_upgrades_csv, item_tables_skyship_upgrades_json)
csv_to_json(Item_Tables_WEAPONS, Item_Tables_WEAPONS_json)



In [ ]:
# Définir le schéma
SCHEMA = {
    "Name": str,
    "Power Cost": str,
    "Description": str,
    "Special Mechanic": str,
    "Duration": int,
    "Primary Power type": str,
    "Secondary power type": str,
    "Arcana Boost": str,
    "Special retrictions": str,
    "speed": int,
    "Rarity": str,
    "Animation": str,
    "Particle Effect": str,
    "Creator": str
}

def load_json_to_dataframe(json_file_path, schema):
    # Charger les données JSON du fichier
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    
    # Vérifier si les clés du schéma existent dans chaque entrée
    for entry in data:
        for key in schema.keys():
            if key not in entry:
                raise ValueError(f"Missing required field: {key}")
    
    # Créer le DataFrame pandas en utilisant le schéma spécifié
    df = pd.DataFrame(data)
    
    # Convertir les types de données selon le schéma
    for column, data_type in schema.items():
        df[column] = df[column].astype(data_type)
    
    return df

# Exemple d'utilisation
df = load_json_to_dataframe("data.json", SCHEMA)
df.head()

In [ ]:
collection_name = 'Abilities Collection'

def upload_data_to_firestore(json_file_path, collection_name):
    # Initialize Firebase Admin SDK
    cred = credentials.Certificate("path/to/serviceAccountKey.json")
    firebase_admin.initialize_app(cred)
    
    # Initialize Firestore client
    db = firestore.client()
    
    # Load JSON data from file
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    
    # Upload data to Firestore
    for entry in data:
        db.collection(collection_name).add(entry)

# Example usage
upload_data_to_firestore("game_abilities.json", "Abilities Collection")
